In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.simplefilter("ignore")

import transformers as tsf
import torch

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import *

In [3]:
# Read review json in chunks
review_itr = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json",
                          lines=True,
                          orient='columns',
                          chunksize=1000000)

In [4]:
# consider only first chunk of size 1M and break

for review in review_itr:
    review_df = review
    break

In [5]:
# Load business json
business_df = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json",
                           lines=True,
                           orient='columns')

In [6]:
# check size

business_df.shape

In [7]:
# distribution of open businesses

business_df.is_open.value_counts()

In [8]:
# review count summary

business_df.review_count.describe()

In [9]:
# Only consider businesses which are 1) Open, 2) review count betn 200-300, 3) are restaurants

business_df = business_df[(business_df.is_open==1)\
            & (business_df.review_count>=200)\
            & (business_df.review_count<=300)\
             & (business_df.categories.str.contains('Restaurant',na=False))]

business_df.shape

In [10]:
# extract price range for each of these businesses

business_df['Price_Range'] = business_df['attributes'].apply(lambda x:x.get('RestaurantsPriceRange2',None))

# check few records
business_df.head()

In [11]:
# check unique price ranges
business_df['Price_Range'].unique()

In [12]:
# remove businesses where price range is not available

business_df = business_df[~business_df['Price_Range'].isna()]
business_df.shape

In [14]:
# Subset reviews data for only businesses that we want and check size

review_df = review_df[review_df.business_id.isin(business_df.business_id.values.tolist())]
review_df.shape

In [15]:
# take the unique business ids which were actually available in reviews data

unq_bus_ids = review_df[review_df.business_id.isin(business_df.business_id.values.tolist())]['business_id'].unique()

In [16]:
# subset business data for those unique business ids

business_df = business_df[business_df.business_id.isin(unq_bus_ids)]
business_df.shape

In [18]:
# finally merge the reviews and business data for our unique set of business ids

review_df = review_df.merge(
    business_df[['business_id','Price_Range']],
    how='left',
    left_on='business_id',
    right_on='business_id')

In [19]:
# check few records
review_df.head()

In [20]:
# check the distribution of price range in reviews data
review_df.Price_Range.value_counts()

In [21]:
# Only consider sample records for each of our price range and create new df

final_df = pd.concat([
    review_df[review_df.Price_Range=='1'].sample(n=4000,random_state=123),
    review_df[review_df.Price_Range=='2'].sample(n=4000,random_state=123),
    review_df[review_df.Price_Range=='3'].sample(n=4000,random_state=123),
    review_df[review_df.Price_Range=='4']
])

In [ ]:
final_df.head()

In [ ]:
# write it to a file for later use

final_df.to_csv('Yelp_task1_data.csv',index=False)

### Read data

In [9]:
import nltk
import random
import string
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
import re       
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [10]:
import pandas as pd
import numpy as np

final_df = pd.read_csv('../input/yelp-task1-data/Yelp_task1_data.csv')

In [11]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [12]:
# lower case our text data

final_df['cleaned_text'] = final_df['text'].apply(lambda x:clean_text(x))

In [13]:
final_df.head(2)

In [14]:
#Split data into train and test

Xtrain,Xtest,y_train,y_test = train_test_split( final_df['cleaned_text'],
                                                final_df['Price_Range'],
                                                test_size=0.25, # 25% will be test size
                                                random_state=123
                                              )

In [33]:
# Create Pipeline to perform series of operation on data
# 1) Count Vectorization, 2) TF-IDF transformation, 3) Classification

pr_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(solver='liblinear',class_weight='balanced')),
#     ('clf',KNeighborsClassifier())
#     ('clf',RandomForestClassifier(max_features=3,class_weight='balanced'))
#     ('clf',SVC(kernel='rbf', class_weight='balanced'))
    ])

In [35]:
param_grid = {
    'vect__max_features':[1000,5000,7500,10000],
    'vect__ngram_range' : [(1,2),(1,3)],
    'clf__C': np.linspace(0.0001, 100, 10),
#     'clf__n_neighbors':[3,5]
#     'clf__max_depth':[3,5,8],
#     'clf__n_estimators':[10,100,250]
}

search = GridSearchCV(pr_clf, param_grid, n_jobs=-1)

search.fit(Xtrain,y_train)

In [36]:
print('best parameters: ', search.best_params_)
print('best scrores: ', search.best_score_)

In [37]:
predicted = search.predict(Xtest)
print(classification_report(y_test,predicted))

In [38]:
# Functional API for predicting price range

def infer_api(reviews):
    return search.predict(reviews)

In [40]:
# predict on sample text

infer_api(final_df['text'].values[:3])

In [41]:
# actual price range

final_df['Price_Range'].values[:3]

### Using BERT based embeddings

In [13]:
# Load pretrained model/tokenizer
tokenizer = tsf.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = tsf.DistilBertModel.from_pretrained('distilbert-base-uncased')

In [8]:
final_df.Price_Range.value_counts()

In [14]:
# change datatype for price range to string
final_df.Price_Range = final_df.Price_Range.astype(str)

In [32]:
# consider only sample data for each price range as BERT based models are memory intensive

sample_df = pd.concat([
    final_df[final_df.Price_Range=='1'].sample(n=1000,random_state=123),
    final_df[final_df.Price_Range=='2'].sample(n=1000,random_state=123),
    final_df[final_df.Price_Range=='3'].sample(n=1000,random_state=123),
    final_df[final_df.Price_Range=='4']#.sample(n=700,random_state=123)
])

In [ ]:
# set the max sequence length

max_len = 100

In [33]:
#Tokenize the text data -> add special tokens (CLS, SEP) -> truncate sequence length to predefined max length

inputs = sample_df['cleaned_text'].apply((lambda x:tokenizer.encode(x,
                                                                    add_special_tokens=True,
                                                                    truncation=True,
                                                                    max_length=max_len)
                                         ))

In [35]:
# add padding to handle shorter sequence

padded = np.array([i + [0]*(max_len-len(i)) for i in inputs.values])
np.array(padded).shape

In [36]:
# create attention mask to distinguish tokens and padding

attention_mask = np.where(padded != 0,1,0)
attention_mask.shape

In [37]:
# convert numpy arrays to torch tensors

input_ids = torch.tensor(padded)#.cuda()
attention_mask = torch.tensor(attention_mask)#.cuda()

In [39]:
# release memory

import gc

del inputs
del padded

gc.collect()

# torch.cuda.empty_cache()

In [40]:
# Run BERT model on our sentences

with torch.no_grad():
    output = model(input_ids,attention_mask)

In [41]:
# In order fetch bert based embeddings for each of our reviews , 
# we need to extract features of CLS token from last hidden states
# [:,0,:] -> [ #reviews, max sequence length, embeddings]

features = output[0][:,0,:].numpy()

In [26]:
# fetch price range for each reviews in labels -> [#reviews,]

labels = sample_df['Price_Range'].values

In [27]:
# check shape of features -> [#reviews, #embedding dims]
features.shape

In [28]:
# train test split data
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.25)

### Try grid search on Logistic Regression

In [29]:
np.linspace(0.0001, 100, 10)

In [54]:
parameters = {'C': np.linspace(0.0001, 100, 10)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(X_train, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

In [31]:
# fit logistic model with best params
clf = LogisticRegression(C=grid_search.best_params_['C'])
clf.fit(X_train, y_train)

# predict on test set and print metric
predicted = clf.predict(X_test)
print(classification_report(y_test,predicted))

### Try PCA to reduce dimensionality from 768 to lesser values

In [43]:
from sklearn.decomposition import PCA

In [49]:
pca = PCA()

# logistic = LogisticRegression()
svc = SVC(kernel='rbf',class_weight='balanced')
pipe = Pipeline(steps=[
    ('pca', pca),
    ('svc',svc)
#     ('logistic', logistic),
])

In [52]:
param_grid = {
    'pca__n_components': [32,64,128,256,512],
#     'logistic__C': np.linspace(0.0001, 100, 10),
}

In [53]:
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(X_train, y_train)

print('best parameters: ', search.best_params_)
print('best scrores: ', search.best_score_)